In [112]:
import numpy as np
import pandas as pd
import os
import unicodedata

## Data Loading
### focus on journals related to finance, economics, management, etc., specifically:
<ol>
  <li>from CABS file use all journals in spreadsheet “CABS 2021”</li>
  <li>from JCR files use all journals in spreadsheets: BUSINESS, BUSINESS, FINANCE, ECONOMICS, MANAGEMENT
</li>
  <li>from “Wykaz…” files use the journals from the following:
ekonomia i finanse		nauki o zarządzaniu i jakości
501		506;
Use only spreadsheet “czasopisma naukowe”
</li>
</ol>

Note that if the journal appears at least in one of the source, it has to be taken into account


In [113]:
# Specify the directory containing your Excel files
directory_path = 'source files'

# Get a list of all Excel files in the directory
excel_files = [file for file in os.listdir(directory_path) if file.endswith('.xlsx')]

# Create dictionaries to store loaded data frames for CABS, JCR, and WYKAZ
cabs_data = {}
jcr_data = {}
wykaz_data = {}

# Load from CABS file
cabs_file = [file for file in excel_files if "CABS" in file][0]
cabs_sheet = "CABS 2021"
cabs_file_path = os.path.join(directory_path, cabs_file)
try:
    cabs_df = pd.read_excel(cabs_file_path, sheet_name=cabs_sheet)
    cabs_data[cabs_file] = cabs_df
except Exception as e:
    print(f"Error loading {cabs_file}: {e}")

# Load from JCR files
jcr_files = [file for file in excel_files if "JCR" in file]
for jcr_file in jcr_files:
    jcr_file_path = os.path.join(directory_path, jcr_file)
    try:
        jcr_data[jcr_file] = pd.read_excel(jcr_file_path, sheet_name=None)
    except Exception as e:
        print(f"Error loading {jcr_file}: {e}")

# Load from files containing the word “Wykaz”
wykaz_files = [file for file in excel_files if "Wykaz" in file]
for wykaz_file in wykaz_files:
    wykaz_file_path = os.path.join(directory_path, wykaz_file)
    try:
        wykaz_df = None
        with pd.ExcelFile(wykaz_file_path) as xls:
            for sheet_name in xls.sheet_names:
                if "czasopisma" in sheet_name.lower():
                    wykaz_df = pd.read_excel(wykaz_file_path, sheet_name=sheet_name, engine='openpyxl')

                    wykaz_data[wykaz_file] = wykaz_df
                    break
        if wykaz_df is None:
            print(f"No sheet containing 'czasopisma' found in {wykaz_file}")
    except Exception as e:
        print(f"Error loading {wykaz_file}: {e}")

# Print the dictionaries
print("CABS Data:")
print(cabs_data.keys())

print("\nJCR Data:")
print(jcr_data.keys())

print("\nWYKAZ Data:")
print(wykaz_data.keys())

CABS Data:
dict_keys(['CABS Journal Ranking 2021.xlsx'])

JCR Data:
dict_keys(['JCR_2021_ALL.xlsx', 'JCR_2022_ALL.xlsx'])

WYKAZ Data:
dict_keys(['20211201_Wykaz_dyscyplin_przypisanych_do_czasopism_naukowych_i_materiałów_konferencyjnych - Dec 2021 1.xlsx', '20240105_Wykaz_czasopism_naukowych_2024_styczeń.xlsx', 'Wykaz_dyscyplin_do_czasopism_i_materiałów_konferencyjnych Feb 2021.xlsx'])


In [114]:
# Drop 'ALL' key from jcr_data['JCR_2022_ALL.xlsx']
if 'JCR_2022_ALL.xlsx' in jcr_data:
    jcr_data['JCR_2022_ALL.xlsx'].pop('ALL', None)

In [115]:
# Replace preceeding column names with true column names
def process_dataframes(dictionary_name, data_dict):
    for file_name, data_frame in data_dict.items():
        # Check if 'issn' is present in any column name (case-insensitive)
        has_issn_column = any('issn' in str(col).lower() for col in data_frame.columns)

        if not has_issn_column:
            # Make the first row the column names
            data_frame.columns = data_frame.iloc[0]
            data_frame = data_frame[1:]  # Exclude the first row after using it as column names

            # Reset the index after excluding the first row
            data_frame.reset_index(drop=True, inplace=True)

            # Update the dataframe in the dictionary
            data_dict[file_name] = data_frame

    print(f"Processed {dictionary_name} data frames.")

process_dataframes('WYKAZ', wykaz_data)


Processed WYKAZ data frames.


In [116]:
# Rename duplicate and inconsistent column names
def rename_duplicate_columns_in_dict(data_dict):
    for df_name, df in data_dict.items():
        column_count = {}
        new_columns = []

        for column in df.columns:
            if column not in column_count:
                column_count[column] = 1
                new_column = column
            else:
                column_count[column] += 1
                new_column = f"{column}{column_count[column]}"

            new_columns.append(new_column)

        df.columns = new_columns

        # Change 'issn.1' to 'issn2' and 'e-issn.1' to 'e-issn2'
        if 'issn.1' in df.columns:
            df.rename(columns={'issn.1': 'issn2', 'e-issn.1': 'e-issn2'}, inplace=True)

        # Change 'Punktacja' to 'Punkty'
        if 'Punktacja' in df.columns:
            df.rename(columns={'Punktacja': 'Points Jan24'}, inplace=True)

        data_dict[df_name] = df

    return data_dict

# Call the function to rename duplicate columns in all DataFrames
rename_duplicate_columns_in_dict(wykaz_data)


{'20211201_Wykaz_dyscyplin_przypisanych_do_czasopism_naukowych_i_materiałów_konferencyjnych - Dec 2021 1.xlsx':          Lp.  Unikatowy Identyfikator Czasopisma  \
 0          1                                   1   
 1          2                                   2   
 2          3                                   3   
 3          4                                   4   
 4          5                                   5   
 ...      ...                                 ...   
 32671  32672                              201488   
 32672  32673                              201489   
 32673  32674                              201490   
 32674  32675                              201491   
 32675  32676                              201492   
 
                                                  Tytuł 1       issn  \
 0                                           2D Materials  2053-1583   
 1                                              3 Biotech  2190-572X   
 2                                 

In [117]:
# Rename Punkty columns to reflect date
def rename_columns_in_wykaz_data(data_dict):
    for df_name, df in data_dict.items():
        # Check if the DataFrame is the specific one
        if df_name == "20211201_Wykaz_dyscyplin_przypisanych_do_czasopism_naukowych_i_materiałów_konferencyjnych - Dec 2021 1.xlsx":
            # Check if 'Punkty' column exists
            if 'Punkty' in df.columns:
                # Rename 'Punkty' column
                df.rename(columns={'Punkty': 'Points Dec21'}, inplace=True)
        elif df_name == "Wykaz_dyscyplin_do_czasopism_i_materiałów_konferencyjnych Feb 2021.xlsx":
            # Check if 'Punkty' column exists
            if 'Punkty' in df.columns:
                # Rename 'Punkty' column
                df.rename(columns={'Punkty': 'Points Feb21'}, inplace=True)

    return data_dict

wykaz_data = rename_columns_in_wykaz_data(wykaz_data)

### Wykaz Files
Extracting the required data to create a united dataframe out of all files

In [118]:
# Create a single DataFrame from all Wykaz files
wykaz_df = pd.DataFrame(columns=['Tytuł 1', 'issn', 'e-issn', 'Tytuł 2', 'issn2', 'e-issn2', 501, 506])

# Extract data from Wykaz files for categories 501 and 506
for file_name, file_data in wykaz_data.items():
    for column in file_data.columns:
        if column in [501, 506]:
            # Find all 'Points' columns in the DataFrame
            points_columns = [col for col in file_data.columns if isinstance(col, str) and col.startswith('Points')]

            # Create the list of selected columns
            selected_columns = ['Tytuł 1', 'issn', 'e-issn', 'Tytuł 2', 'issn2', 'e-issn2', 501, 506] + points_columns

            # Print the actual column names in the DataFrame
            print(f"Actual Column Names in {file_name}: {file_data.columns.tolist()}")

            selected_data = file_data[file_data[column] == 'x'][selected_columns]

            # Print the selected data to see if the columns are present
            print(f"Selected Data for {file_name}:\n{selected_data.head()}")

            wykaz_df = pd.concat([wykaz_df, selected_data])

# Deduplicate the resulting DataFrame
wykaz_df = wykaz_df.drop_duplicates()

# Replace various representations of missing values in 'issn' and 'e-issn' with NaN
missing_values = ['', '         ', 'NA', 'N/A', 'NaN', 'nan', None]
wykaz_df['issn'].replace(missing_values, np.nan, inplace=True)
wykaz_df['e-issn'].replace(missing_values, np.nan, inplace=True)

# Fill NaN values in 'Tytuł 1', 'issn', 'e-issn' with values from 'Tytuł 2', 'issn2', 'e-issn2' respectively
wykaz_df['Tytuł 1'].fillna(wykaz_df['Tytuł 2'], inplace=True)
wykaz_df['issn'].fillna(wykaz_df['issn2'], inplace=True)
wykaz_df['e-issn'].fillna(wykaz_df['e-issn2'], inplace=True)

# Drop the 'Tytuł 2', 'issn2', 'e-issn2' columns
wykaz_df = wykaz_df.drop(columns=['Tytuł 2', 'issn2', 'e-issn2'])

# Display the resulting Wykaz DataFrame
print("Wykaz DataFrame:")
wykaz_df

Actual Column Names in 20211201_Wykaz_dyscyplin_przypisanych_do_czasopism_naukowych_i_materiałów_konferencyjnych - Dec 2021 1.xlsx: ['Lp.', ' Unikatowy Identyfikator Czasopisma', 'Tytuł 1', 'issn', 'e-issn', 'Tytuł 2', 'issn2', 'e-issn2', 'Points Dec21', 101, 102, 103, 104, 105, 106, 107, 201, 202, 203, 204, 205, 206, 207, 208, 209, 301, 302, 303, 304, 401, 402, 403, 404, 405, 501, 502, 503, 504, 505, 506, 507, 508, 509, 510, 511, 601, 602, 603, 604, 605, 606, 607, 701]
Selected Data for 20211201_Wykaz_dyscyplin_przypisanych_do_czasopism_naukowych_i_materiałów_konferencyjnych - Dec 2021 1.xlsx:
                                              Tytuł 1       issn     e-issn  \
9      4OR-A Quarterly Journal of Operations Research  1619-4500  1614-2411   
22  Abacus-A Journal of Accounting Finance and Bus...  0001-3072  1467-6281   
54                             Accounting and Finance  0810-5391  1467-629X   
56           Accounting Economics and Law-A Convivium  2194-6051  2152-2820   
67 

Selected Data for Wykaz_dyscyplin_do_czasopism_i_materiałów_konferencyjnych Feb 2021.xlsx:
                                                 Tytuł 1       issn  \
27508     4OR-A Quarterly Journal of Operations Research  1619-4500   
27509  Abacus-A Journal of Accounting Finance and Bus...  0001-3072   
27510                             Accounting and Finance  0810-5391   
27511           Accounting Economics and Law-A Convivium  2194-6051   
27512                                  ACCOUNTING REVIEW  0001-4826   

          e-issn                        Tytuł 2      issn2    e-issn2 501 506  \
27508  1614-2411                            4OR  1619-4500  1614-2411   x   x   
27509  1467-6281                         Abacus  0001-3072        NaN   x   x   
27510  1467-629X         Accounting and Finance  0810-5391        NaN   x   x   
27511  2152-2820  Accounting, Economics and Law        NaN  2152-2820   x   x   
27512  1558-7967              Accounting Review  0001-4826        NaN   x   x

,Tytuł 1,issn,e-issn,501,506,Points Dec21,Points Jan24,Points Feb21
9,4OR-A Quarterly Journal of Operations Research,1619-4500,1614-2411,x,x,70,NaN,NaN
22,Abacus-A Journal of Accounting Finance and Bus...,0001-3072,1467-6281,x,x,70,NaN,NaN
54,Accounting and Finance,0810-5391,1467-629X,x,x,70,NaN,NaN
56,Accounting Economics and Law-A Convivium,2194-6051,2152-2820,x,x,70,NaN,NaN
67,ACCOUNTING REVIEW,0001-4826,1558-7967,x,x,200,NaN,NaN
...,...,...,...,...,...,...,...,...
31208,Internetowy Kwartalnik Antymonopolowy i Regula...,2299-8837,2299-5749,NaN,x,NaN,NaN,20.0
31235,Humanum. MIędzynarodowe Studia Społeczno-Human...,1898-8431,2450-0313,NaN,x,NaN,NaN,20.0
31259,Problemy Jakości,0137-8651,2449-9862,NaN,x,NaN,NaN,20.0
31277,Społeczeństwo. Edukacja. Język,2353-1266,NaN,NaN,x,NaN,NaN,40.0


In [119]:
# Replace '*' with 1 in columns 501 and 506
wykaz_df[501] = wykaz_df[501].replace('x', 1)
wykaz_df[506] = wykaz_df[506].replace('x', 1)

# Convert columns to categorical type
wykaz_df[501] = wykaz_df[501].astype('category')
wykaz_df[506] = wykaz_df[506].astype('category')

# Rename columns
wykaz_df = wykaz_df.rename(columns={501: 'ekonomia i finance', 506: 'nauki o zarządzaniu i jakości'})

# Display the resulting DataFrame
print("Updated wykaz_df:")
print(wykaz_df)

Updated wykaz_df:
                                                 Tytuł 1       issn  \
9         4OR-A Quarterly Journal of Operations Research  1619-4500   
22     Abacus-A Journal of Accounting Finance and Bus...  0001-3072   
54                                Accounting and Finance  0810-5391   
56              Accounting Economics and Law-A Convivium  2194-6051   
67                                     ACCOUNTING REVIEW  0001-4826   
...                                                  ...        ...   
31208  Internetowy Kwartalnik Antymonopolowy i Regula...  2299-8837   
31235  Humanum. MIędzynarodowe Studia Społeczno-Human...  1898-8431   
31259                                   Problemy Jakości  0137-8651   
31277                     Społeczeństwo. Edukacja. Język  2353-1266   
31406                                   Management Forum        NaN   

          e-issn ekonomia i finance nauki o zarządzaniu i jakości  \
9      1614-2411                1.0                         

In [120]:
wykaz_df['e-issn'] = wykaz_df['e-issn'].fillna(wykaz_df['issn'])

# Deduplicate wykaz_df based on 'Tytuł 1', 'issn', 'e-issn'
wykaz_df['Tytuł 1'] = wykaz_df['Tytuł 1'].str.strip()
wykaz_df['Tytuł 1'] = wykaz_df['Tytuł 1'].str.encode('utf-8').str.decode('utf-8')
wykaz_df['Tytuł 1'] = wykaz_df['Tytuł 1'].apply(lambda x: unicodedata.normalize('NFKD', x))
wykaz_df['Tytuł 1'] = wykaz_df['Tytuł 1'].str.replace(r'\s+', ' ', regex=True)
wykaz_df_deduplicated = wykaz_df.groupby(['Tytuł 1', 'issn', 'e-issn']).apply(lambda group: group.ffill().bfill())
wykaz_df_deduplicated = wykaz_df.groupby(['issn', 'e-issn']).apply(lambda group: group.ffill().bfill())

# Reset index to have a clean DataFrame
wykaz_df_deduplicated.reset_index(drop=True, inplace=True)

# Drop fully duplicated rows
wykaz_df_deduplicated = wykaz_df_deduplicated.drop_duplicates(subset=['issn', 'e-issn'])

wykaz_df_deduplicated.rename(columns={'Tytuł 1': 'Title'}, inplace=True)

# Display the resulting deduplicated Wykaz DataFrame
print("Deduplicated Wykaz DataFrame:")
print(wykaz_df_deduplicated)

C:\Users\Alon\AppData\Local\Temp\ipykernel_8404\3708523349.py:8: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  wykaz_df_deduplicated = wykaz_df.groupby(['Tytuł 1', 'issn', 'e-issn']).apply(lambda group: group.ffill().bfill())
C:\Users\Alon\AppData\Local\Temp\ipykernel_8404\3708523349.py:9: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use

Deduplicated Wykaz DataFrame:
                                                  Title       issn     e-issn  \
0        4OR-A Quarterly Journal of Operations Research  1619-4500  1614-2411   
2     Abacus-A Journal of Accounting Finance and Bus...  0001-3072  1467-6281   
4                                      Accounting Forum  0155-9982  1467-6303   
5                                Accounting and Finance  0810-5391  1467-629X   
7                                    Accounting History  1032-3732  1749-3374   
...                                                 ...        ...        ...   
9759             Journal of Tourism Management Research  2408-9117  2313-4178   
9760   Journal of Tourism Sustainability and Well-being  2795-5044  2795-5044   
9761                Research in Organizational Behavior  0191-3085  0191-3085   
9762             Recherche et Applications en Marketing  0767-3701  2051-2821   
9763            Przegląd Bezpieczeństwa Wewnętrznego  2080-1335  2720-0841  

### JCR Files
Extracting the required data to create a united dataframe out of all files

In [122]:
for file_name, year_data in jcr_data.items():
    year_suffix = file_name.split('_')[1].split('.')[0]
    year_suffix_text = f' (in {year_suffix})'

    for df_name, df in year_data.items():
        # Change column names
        df.columns = df.columns.str.replace('ISSN', 'issn', case=False)
        df.columns = df.columns.str.replace('eISSN', 'e-issn', case=False)
        df.rename(columns={'Journal name': 'Tytuł 1'}, inplace=True)

        # Add suffix to columns not mentioned
        for col in df.columns:
            if col.lower() not in ['issn', 'e-issn', 'tytuł 1', 'category']:
                df.rename(columns={col: col + year_suffix_text}, inplace=True)

In [123]:
# List of columns to include in jcr_df
columns_to_include = [
    'Tytuł 1', 'issn', 'e-issn', 'Category', '2021 JIF (in 2021)', 'JIF Quartile (in 2021)',
    '5 Year JIF (in 2021)', 'JIF Without Self Cites (in 2021)', 'JIF Percentile (in 2021)',
    '2021 JCI (in 2021)', 'JCI Percentile (in 2021)',
    '2022 JIF (in 2022)', 'JIF Quartile (in 2022)', '5 Year JIF (in 2022)',
    '5 Year JIF Quartile (in 2022)', 'JIF Rank (in 2022)', '2022 JCI (in 2022)',
    'JCI Quartile (in 2022)', 'JCI Rank (in 2022)'
]

# Create an empty DataFrame for jcr_df
jcr_df = pd.DataFrame(columns=columns_to_include)

# Concatenate DataFrames for each year
for year, year_data in jcr_data.items():
    for df_name, df in year_data.items():
        print(f"Columns for {df_name} ({year}): {df.columns}")

        # Filter columns to include only the specified ones
        existing_columns = [col for col in columns_to_include if col in df.columns]
        df_subset = df[existing_columns].copy()

        # If a column is missing, fill it with NA
        missing_columns = [col for col in columns_to_include if col not in df.columns]
        for col in missing_columns:
            df_subset[col] = np.nan

        # Concatenate to jcr_df
        jcr_df = pd.concat([jcr_df, df_subset], ignore_index=True)

# Display the resulting DataFrame
print("jcr_df:")
print(jcr_df)


Columns for BUSINESS (JCR_2021_ALL.xlsx): Index(['Tytuł 1', 'issn', 'e-issn', 'Category', 'Total Citations (in 2021)',
       '2021 JIF (in 2021)', 'JIF Quartile (in 2021)', '2021 JCI (in 2021)',
       '% of OA Gold (in 2021)', '5 Year JIF (in 2021)',
       'Immediacy Index (in 2021)', 'JIF Without Self Cites (in 2021)',
       'Eigenfactor (in 2021)', 'Normalized Eigenfactor (in 2021)',
       'Article Influence Score (in 2021)', 'JIF Percentile (in 2021)',
       'Total Articles (in 2021)', 'Citing Half-Life (in 2021)',
       'Cited Half-Life (in 2021)', 'AIS Quartile (in 2021)',
       'Citable Items (in 2021)', '% of Articles in Citable items (in 2021)',
       'JCI Percentile (in 2021)'],
      dtype='object')
Columns for BUSINESS, FINANCE (JCR_2021_ALL.xlsx): Index(['Tytuł 1', 'issn', 'e-issn', 'Category', 'Total Citations (in 2021)',
       '2021 JIF (in 2021)', 'JIF Quartile (in 2021)', '2021 JCI (in 2021)',
       '% of OA Gold (in 2021)', '5 Year JIF (in 2021)',
       'Im

In [124]:
jcr_df = jcr_df.groupby(['issn', 'Category']).apply(lambda group: group.ffill().bfill())

C:\Users\Alon\AppData\Local\Temp\ipykernel_8404\177033880.py:1: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  jcr_df = jcr_df.groupby(['issn', 'Category']).apply(lambda group: group.ffill().bfill())


In [125]:
# Find duplicate rows based on 'issn' and 'Category'
duplicate_condition = jcr_df.duplicated(subset=['issn', 'e-issn', 'Category'], keep=False)

# Extract rows with duplicates
duplicates_df_jcr = jcr_df[duplicate_condition]

duplicates_df_jcr = duplicates_df_jcr.sort_values(by=['issn', 'e-issn', 'Category'])

duplicates_df_jcr = duplicates_df_jcr.reset_index(drop=True)

# Display the DataFrame with duplicate rows
print("DataFrame with duplicate rows based on 'issn' and 'Category' (sorted by 'issn' and 'Category'):")
print(duplicates_df_jcr)

DataFrame with duplicate rows based on 'issn' and 'Category' (sorted by 'issn' and 'Category'):
                                                Tytuł 1       issn     e-issn  \
0     Abacus-A Journal of Accounting Finance and Bus...  0001-3072  1467-6281   
1     Abacus-A Journal of Accounting Finance and Bus...  0001-3072  1467-6281   
2                         ACADEMY OF MANAGEMENT JOURNAL  0001-4273  1948-0989   
3                         ACADEMY OF MANAGEMENT JOURNAL  0001-4273  1948-0989   
4                         ACADEMY OF MANAGEMENT JOURNAL  0001-4273  1948-0989   
...                                                 ...        ...        ...   
4772                           ISSUES IN LAW & MEDICINE  8756-8160        NaN   
4773           JOURNAL OF COLLEGE STUDENT PSYCHOTHERAPY  8756-8225  1540-4730   
4774           JOURNAL OF COLLEGE STUDENT PSYCHOTHERAPY  8756-8225  1540-4730   
4775                         Project Management Journal  8756-9728  1938-9507   
4776         

In [126]:
# Make sure all duplicates in duplicates_df_jcr are pairs of 2021-2022 with corresponding NaN values

all_nan_for_all_odd_indexes = True
failed_indexes = []

# Iterate through all odd-numbered indexes
for i in range(1, len(duplicates_df_jcr), 2):
    # Get the current row
    current_row = duplicates_df_jcr.iloc[i]

    # Check if the specified columns are all NaN
    if current_row[['2021 JIF (in 2021)', 'JIF Quartile (in 2021)', '5 Year JIF (in 2021)',
                    'JIF Without Self Cites (in 2021)', 'JIF Percentile (in 2021)',
                    '2021 JCI (in 2021)', 'JCI Percentile (in 2021)']].isna().all():
        print(f"Columns are all NaN for index {i}: {current_row[['issn', 'Category']]}")
    else:
        all_nan_for_all_odd_indexes = False
        failed_indexes.append(i)

# Print a message based on whether all columns were NaN for all odd indexes
if all_nan_for_all_odd_indexes:
    print("Yes, all specified columns are NaN for all odd indexes.")
else:
    print("No, some specified columns are not all NaN for at least one odd index.")
    print("Indexes where the rule fails:", failed_indexes)


Columns are all NaN for index 23: issn         0002-4821
Category    LAW - ESCI
Name: 23, dtype: object
Columns are all NaN for index 41: issn         0002-9319
Category    LAW - ESCI
Name: 41, dtype: object
Columns are all NaN for index 61: issn         0004-9611
Category    LAW - ESCI
Name: 61, dtype: object
Columns are all NaN for index 79: issn         0007-974X
Category    LAW - ESCI
Name: 79, dtype: object
Columns are all NaN for index 203: issn         0018-9855
Category    LAW - ESCI
Name: 203, dtype: object
Columns are all NaN for index 235: issn         0021-2237
Category    LAW - ESCI
Name: 235, dtype: object
Columns are all NaN for index 237: issn         0021-325X
Category    LAW - ESCI
Name: 237, dtype: object
Columns are all NaN for index 273: issn         0022-0183
Category    LAW - ESCI
Name: 273, dtype: object
Columns are all NaN for index 411: issn         0030-6185
Category    LAW - ESCI
Name: 411, dtype: object
Columns are all NaN for index 435: issn         0033-7

In [127]:
# Deduplicate rows including rows with missing identifiers for issn or e-issn
# Define boolean masks for issn and e-issn duplicates
issn_duplicates_mask = jcr_df.duplicated(subset=['issn', 'Category'], keep=False)
eissn_duplicates_mask = jcr_df.duplicated(subset=['e-issn', 'Category'], keep=False)

# Combine rows for issn duplicates
jcr_df_issn_combined = jcr_df[issn_duplicates_mask].groupby(['Tytuł 1', 'issn', 'Category'], as_index=False).apply(lambda x: x.combine_first(x.shift(-1))).reset_index(drop=True)

# Combine rows for e-issn duplicates
jcr_df_eissn_combined = jcr_df[eissn_duplicates_mask].groupby(['Tytuł 1', 'e-issn', 'Category'], as_index=False).apply(lambda x: x.combine_first(x.shift(-1))).reset_index(drop=True)

# Keep unique rows
unique_rows = jcr_df[~(issn_duplicates_mask | eissn_duplicates_mask)]

# Concatenate the deduplicated and unique rows
jcr_df_combined = pd.concat([jcr_df_issn_combined, jcr_df_eissn_combined, unique_rows], ignore_index=True)

# Drop the original duplicate rows
jcr_df_combined = jcr_df_combined.drop_duplicates(subset=['Tytuł 1', 'issn', 'e-issn', 'Category'])

# Display the resulting combined DataFrame
print("Combined jcr_df:")
print(jcr_df_combined.head())


C:\Users\Alon\AppData\Local\Temp\ipykernel_8404\3447710790.py:7: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  jcr_df_issn_combined = jcr_df[issn_duplicates_mask].groupby(['Tytuł 1', 'issn', 'Category'], as_index=False).apply(lambda x: x.combine_first(x.shift(-1))).reset_index(drop=True)
C:\Users\Alon\AppData\Local\Temp\ipykernel_8404\3447710790.py:10: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=Fa

Combined jcr_df:
                                             Tytuł 1       issn     e-issn  \
0                                         3C Empresa  2254-3376  2254-3376   
1  Academia-Revista Latinoamericana de Administra...  1012-8255  2056-5127   
2                       Academy of Management Annals  1941-6520  1941-6067   
3                      ACADEMY OF MANAGEMENT JOURNAL  0001-4273  1948-0989   
4                 Academy of Management Perspectives  1558-9080        NaN   

          Category  2021 JIF (in 2021) JIF Quartile (in 2021)  \
0  BUSINESS - ESCI                 NaN                    NaN   
1  BUSINESS - SSCI               1.369                     Q4   
2  BUSINESS - SSCI              19.241                     Q1   
3  BUSINESS - SSCI              10.979                     Q1   
4  BUSINESS - SSCI               8.069                     Q1   

   5 Year JIF (in 2021)  JIF Without Self Cites (in 2021)  \
0                   NaN                               NaN   
1

In [128]:
# Group by 'issn' and 'Category', and aggregate columns, keeping the first non-NaN value
deduplicated_jcr_df_combined = jcr_df_combined.groupby(['issn', 'Category']).agg(lambda x: x.dropna().iloc[0] if x.notna().any() else np.nan).reset_index()

# Display the deduplicated DataFrame
print("Deduplicated Merged DataFrame:")
deduplicated_jcr_df_combined

Deduplicated Merged DataFrame:


,issn,Category,Tytuł 1,e-issn,2021 JIF (in 2021),JIF Quartile (in 2021),5 Year JIF (in 2021),JIF Without Self Cites (in 2021),JIF Percentile (in 2021),2021 JCI (in 2021),JCI Percentile (in 2021),2022 JIF (in 2022),JIF Quartile (in 2022),5 Year JIF (in 2022),5 Year JIF Quartile (in 2022),JIF Rank (in 2022),2022 JCI (in 2022),JCI Quartile (in 2022),JCI Rank (in 2022)
0,0001-3072,"BUSINESS, FINANCE - SSCI",Abacus-A Journal of Accounting Finance and Bus...,1467-6281,2.060,Q3,3.243,1.800,33.78,0.97,72.62,2.1,Q3,2.6,Q3,69/111,0.66,Q2,96/227
1,0001-4273,BUSINESS - SSCI,ACADEMY OF MANAGEMENT JOURNAL,1948-0989,10.979,Q1,16.178,10.611,90.58,1954-02-01 00:00:00,98.14,10.5,Q1,14.5,Q1,15/154,2.40,Q1,8/301
2,0001-4273,MANAGEMENT - SSCI,ACADEMY OF MANAGEMENT JOURNAL,1948-0989,10.979,Q1,16.178,10.611,93.58,2.54,97.57,10.5,Q1,14.5,Q1,16/227,2.40,Q1,9/396
3,0001-4788,"BUSINESS, FINANCE - SSCI",ACCOUNTING AND BUSINESS RESEARCH,2159-4260,2.326,Q3,3.606,1.953,40.09,1.0,74.43,1.7,Q3,3.2,Q3,78/111,0.70,Q2,90/227
4,0001-4826,"BUSINESS, FINANCE - SSCI",ACCOUNTING REVIEW,1558-7967,5.182,Q1,7.818,4.852,87.84,1.78,92.99,4.1,Q2,6.6,Q1,35/111,1.48,Q1,21/227
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2650,8756-6222,ECONOMICS - SSCI,JOURNAL OF LAW ECONOMICS & ORGANIZATION,1465-7341,1.324,Q3,2.019,1.297,26.78,1.11,79.04,1.1,Q4,1.6,Q3,299/380,0.75,Q2,198/581
2651,8756-6222,LAW - SSCI,JOURNAL OF LAW ECONOMICS & ORGANIZATION,1465-7341,NaN,Q2,2.151,2.050,NaN,NaN,NaN,1.1,Q3,1.6,Q2,88/152,0.75,Q2,165/411
2652,8756-8160,LAW - SSCI,ISSUES IN LAW & MEDICINE,NaN,NaN,Q4,0.226,0.320,NaN,NaN,NaN,0.9,Q3,0.7,Q4,102/152,0.88,Q2,134/411
2653,8756-8225,"PSYCHOLOGY, APPLIED - ESCI",JOURNAL OF COLLEGE STUDENT PSYCHOTHERAPY,1540-4730,NaN,NaN,NaN,NaN,NaN,0.27,15.63,1.4,NaN,1.2,NaN,NaN,0.35,Q4,90/115


### CUBS File
Extracting the required data to prepare a dataframe for concatenation

In [129]:
# Directly create a dataframe from the single file in cabs_data
cabs_df = cabs_data['CABS Journal Ranking 2021.xlsx']

# Clean column names by removing trailing spaces
cabs_df.columns = cabs_df.columns.str.strip()

# Specify the columns and their corresponding new names
columns_mapping = {
    'ISSN': 'e-issn',
    'Field': 'Class',
    'Journal Title': 'Journal Title (CABS)',
    'Publisher Name': 'Publisher Name',
    'AJG 2015': 'AJG 2015',
    'AJG 2018': 'AJG 2018',
    'AJG 2021': 'AJG 2021'
}

# Select and rename columns if they exist in the DataFrame
cabs_df_selected = cabs_df[columns_mapping.keys()].rename(columns=columns_mapping)

# Display the resulting DataFrame
print("CABS DataFrame:")
print(cabs_df_selected.head())

CABS DataFrame:
      e-issn    Class                   Journal Title (CABS)  \
0  1558-7967  ACCOUNT                      Accounting Review   
1  1873-6289  ACCOUNT  Accounting, Organizations and Society   
2  1879-1980  ACCOUNT    Journal of Accounting and Economics   
3  1475-679X  ACCOUNT         Journal of Accounting Research   
4  1911-3846  ACCOUNT       Contemporary Accounting Research   

                    Publisher Name AJG 2015 AJG 2018 AJG 2021  
0  American Accounting Association       4*       4*       4*  
1                         Elsevier       4*       4*       4*  
2                         Elsevier       4*       4*       4*  
3                  Wiley-Blackwell       4*       4*       4*  
4                  Wiley-Blackwell        4        4        4  


### Join All Sources to Get a Single Output File

In [130]:
# Drop 'e-issn' column from wykaz_df_deduplicated
wykaz_df_deduplicated.drop(columns=['e-issn'], inplace=True)

# Perform an outer join based on 'issn'
merged_df = pd.merge(deduplicated_jcr_df_combined, wykaz_df_deduplicated, left_on=['issn'], right_on=['issn'], how='outer', suffixes=('_jcr', '_wykaz'))
# Fill missing values in 'Tytuł 1' with values from 'Title'
merged_df['Tytuł 1'] = merged_df['Tytuł 1'].fillna(merged_df['Title'])

# Fill missing values in 'e-issn' with values from 'issn'
merged_df['e-issn'] = merged_df['e-issn'].fillna(merged_df['issn'])

# Share value of points between all observations
merged_df = merged_df.groupby(['issn', 'e-issn']).apply(lambda group: group.ffill().bfill())

# Deduplicate merged_df
merged_df = merged_df.drop_duplicates()

# Display the resulting merged DataFrame
print("Merged DataFrame:")
merged_df


C:\Users\Alon\AppData\Local\Temp\ipykernel_8404\63875773.py:9: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  merged_df = merged_df.groupby(['issn', 'e-issn']).apply(lambda group: group.ffill().bfill())


Merged DataFrame:


,issn,Category,Tytuł 1,e-issn,2021 JIF (in 2021),JIF Quartile (in 2021),5 Year JIF (in 2021),JIF Without Self Cites (in 2021),JIF Percentile (in 2021),2021 JCI (in 2021),...,JIF Rank (in 2022),2022 JCI (in 2022),JCI Quartile (in 2022),JCI Rank (in 2022),Title,ekonomia i finance,nauki o zarządzaniu i jakości,Points Dec21,Points Jan24,Points Feb21
0,0001-3072,"BUSINESS, FINANCE - SSCI",Abacus-A Journal of Accounting Finance and Bus...,1467-6281,2.060,Q3,3.243,1.800,33.78,0.97,...,69/111,0.66,Q2,96/227,Abacus-A Journal of Accounting Finance and Bus...,1.0,1.0,70.0,70.0,70.0
1,0001-4273,BUSINESS - SSCI,ACADEMY OF MANAGEMENT JOURNAL,1948-0989,10.979,Q1,16.178,10.611,90.58,1954-02-01 00:00:00,...,15/154,2.40,Q1,8/301,ACADEMY OF MANAGEMENT JOURNAL,NaN,1.0,200.0,200.0,200.0
2,0001-4273,MANAGEMENT - SSCI,ACADEMY OF MANAGEMENT JOURNAL,1948-0989,10.979,Q1,16.178,10.611,93.58,2.54,...,16/227,2.40,Q1,9/396,ACADEMY OF MANAGEMENT JOURNAL,NaN,1.0,200.0,200.0,200.0
3,0001-4788,"BUSINESS, FINANCE - SSCI",ACCOUNTING AND BUSINESS RESEARCH,2159-4260,2.326,Q3,3.606,1.953,40.09,1.0,...,78/111,0.70,Q2,90/227,ACCOUNTING AND BUSINESS RESEARCH,NaN,1.0,100.0,100.0,100.0
4,0001-4826,"BUSINESS, FINANCE - SSCI",ACCOUNTING REVIEW,1558-7967,5.182,Q1,7.818,4.852,87.84,1.78,...,35/111,1.48,Q1,21/227,ACCOUNTING REVIEW,1.0,1.0,200.0,200.0,200.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5607,2408-9117,NaN,Journal of Tourism Management Research,2408-9117,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Journal of Tourism Management Research,NaN,1.0,NaN,20.0,NaN
5608,2795-5044,NaN,Journal of Tourism Sustainability and Well-being,2795-5044,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Journal of Tourism Sustainability and Well-being,NaN,1.0,NaN,20.0,NaN
5609,0191-3085,NaN,Research in Organizational Behavior,0191-3085,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Research in Organizational Behavior,NaN,1.0,NaN,70.0,NaN
5610,0767-3701,NaN,Recherche et Applications en Marketing,0767-3701,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Recherche et Applications en Marketing,NaN,1.0,NaN,20.0,NaN


In [131]:
# Check for duplicates based on 'issn' and 'Category'
duplicates_df = merged_df[merged_df.duplicated(subset=['issn', 'Category'], keep=False)]

# Display the rows with duplicates
print("Duplicate Rows based on 'issn' and 'Category':")
print(duplicates_df)


Duplicate Rows based on 'issn' and 'Category':
           issn Category                                            Tytuł 1  \
3923  2451-1935      NaN                    Econometric Research in Finance   
3924  2451-1935      NaN                     Econometric Researc in Finance   
3925  1234-8872      NaN    Studia i Prace Kolegium Zarządzania i Finasów   
3926  1234-8872      NaN   Studia i Prace Kolegium Zarządzania i Finansów   
5455  2080-4075      NaN  Advances in Science and Technology Research Jo...   
5456  2080-4075      NaN  Advances in Science and Technology-Research Jo...   

         e-issn  2021 JIF (in 2021) JIF Quartile (in 2021)  \
3923  2451-1935                 NaN                    NaN   
3924  2451-1935                 NaN                    NaN   
3925  1234-8872                 NaN                    NaN   
3926  1234-8872                 NaN                    NaN   
5455  2080-4075                 NaN                    NaN   
5456  2080-4075            

In [132]:
# Merge CABS file to add classifications
merged_all_df = pd.merge(merged_df, cabs_df_selected, left_on=['e-issn'], right_on=['e-issn'], how='outer')

In [133]:
# Check for duplicates based on 'issn' and 'Category'
duplicates_df = merged_all_df[merged_all_df.duplicated(subset=['issn', 'Category'], keep=False)]

# Display the rows with duplicates
print("Duplicate Rows based on 'issn' and 'Category':")
print(duplicates_df)

Duplicate Rows based on 'issn' and 'Category':
           issn Category                                            Tytuł 1  \
3911  2451-1935      NaN                    Econometric Research in Finance   
3912  2451-1935      NaN                     Econometric Researc in Finance   
3913  1234-8872      NaN    Studia i Prace Kolegium Zarządzania i Finasów   
3914  1234-8872      NaN   Studia i Prace Kolegium Zarządzania i Finansów   
5435  2080-4075      NaN  Advances in Science and Technology Research Jo...   
...         ...      ...                                                ...   
6307        NaN      NaN                                                NaN   
6308        NaN      NaN                                                NaN   
6309        NaN      NaN                                                NaN   
6310        NaN      NaN                                                NaN   
6311        NaN      NaN                                                NaN   

    

In [134]:
# Organise columns for saving
desired_order = [
     'Class','Category', 'issn', 'e-issn', 'Journal Title (CABS)', 'Tytuł 1',
    'AJG 2015', 'AJG 2018', 'AJG 2021', '2022 JIF (in 2022)', 'JIF Quartile (in 2022)', 'JIF Rank (in 2022)',
    '5 Year JIF (in 2022)', '5 Year JIF Quartile (in 2022)', '2022 JCI (in 2022)', 'JCI Rank (in 2022)',
    'JCI Quartile (in 2022)', '2021 JIF (in 2021)', 'JIF Quartile (in 2021)', '5 Year JIF (in 2021)',
    'JIF Without Self Cites (in 2021)', 'JIF Percentile (in 2021)', '2021 JCI (in 2021)', 'JCI Percentile (in 2021)',
    'Points Feb21', 'Points Dec21', 'Points Jan24', 'ekonomia i finance', 'nauki o zarządzaniu i jakości', 'Publisher Name'
]

# Reorder the columns
merged_all_df = merged_all_df[desired_order]

In [135]:
# Perform small cosmetic changes before saving
merged_all_df['Class'].fillna(merged_all_df['Category'], inplace=True) # Fill missing Class values from JCR Category column
# Rename  Category to "JCR Class"
merged_all_df.rename(columns={'Category': 'JCR Class'}, inplace=True)

merged_all_df.sort_values(by=['issn', 'Journal Title (CABS)', 'Tytuł 1'], inplace=True)  # Sort by specified columns


In [136]:
merged_all_df['Class']

4069                     NaN
0                    ACCOUNT
3981                     NaN
1             ETHICS-CSR-MAN
2             ETHICS-CSR-MAN
                ...         
5986                     MKT
6156          PSYCH (WOP-OB)
6104         PSYCH (GENERAL)
5630    BUS HIST & ECON HIST
5883                INFO MAN
Name: Class, Length: 6312, dtype: object

In [137]:
# Clean Class to iterate through it and dynamically save it to Excel
class_counts = merged_all_df['Class'].value_counts().reset_index()
class_counts.columns = ['Class', 'Observations']
print(class_counts)

# Rename and organize classes according to CABS and JCR
class_mapping = {
    'BUSINESS, FINANCE - SSCI': 'BUSINESS, FINANCE',
    'BUSINESS, FINANCE - ESCI': 'BUSINESS, FINANCE',
    'BUSINESS - SSCI': 'BUSINESS',
    'BUSINESS - ESCI': 'BUSINESS',
    'ECONOMICS - SSCI': 'ECONOMICS',
    'ECONOMICS - ESCI': 'ECONOMICS',
    'MANAGEMENT - SSCI': 'MANAGEMENT',
    'MANAGEMENT - ESCI': 'MANAGEMENT',
    'ECON': 'CABS ECON',
    'ETHICS-CSR-MAN': 'CABS ETHICS-CSR-MAN',
    'FINANCE': 'CABS FINANCE',
    'SECTOR': 'CABS SECTOR',
    'INFO MAN': 'CABS INFO MAN',
    'ACCOUNT': 'CABS ACCOUNT',
    'SOC SCI': 'CABS SOC SCI',
    'MKT': 'CABS MKT',
    'OR&MANSCI': 'CABS OR&MANSCI',
    'OPS&TECH': 'CABS OPS&TECH',
    'PSYCH (GENERAL)': 'CABS PSYCH (GENERAL)',
    'PSYCH (WOP-OB)': 'CABS PSYCH (WOP-OB)',
    'HRM&EMP': 'CABS HRM&EMP',
    'PUB SEC': 'CABS PUB SEC',
    'IB&AREA': 'CABS IB&AREA',
    'MDEV&EDU': 'CABS MDEV&EDU',
    'INNOV': 'CABS INNOV',
    'ORG STUD': 'CABS ORG STUD',
    'ENT-SBM': 'CABS ENT-SBM',
    'REGIONAL STUDIES, PLANNING AND ENVIRONMENT': 'CABS REGIONAL STUDIES, PLANNING AND ENVIRONMENT',
    'BUS HIST & ECON HIST': 'CABS BUS HIST & ECON HIST',
    'STRAT': 'CABS STRAT',
    'LAW - ESCI':'LAW',
    'LAW - SSCI':'LAW',
    'SOCIOLOGY - SSCI':'SOCIOLOGY',
    'SOCIOLOGY - ESCI':'SOCIOLOGY',
    'HEALTH POLICY & SERVICES - SSCI':'HEALTH POLICY & SERVICES',
    'HEALTH POLICY & SERVICES - ESCI':'HEALTH POLICY & SERVICES',
    'PUBLIC ADMINISTRATION - SSCI':'PUBLIC ADMINISTRATION',
    'PUBLIC ADMINISTRATION - ESCI':'PUBLIC ADMINISTRATION', 
    'CULTURAL STUDIES - ESCI':'CULTURAL STUDIES',
    'CULTURAL STUDIES - SSCI':'CULTURAL STUDIES',
    'PSYCHOLOGY, SOCIAL - ESCI' : 'PSYCHOLOGY, SOCIAL',
    'PSYCHOLOGY, SOCIAL - SSCI' : 'PSYCHOLOGY, SOCIAL',
    'PSYCHOLOGY, APPLIED - SSCI' : 'PSYCHOLOGY, APPLIED',
    'PSYCHOLOGY, APPLIED - ESCI' : 'PSYCHOLOGY, APPLIED',
    'CULTURAL STUDIES - AHCI' : 'CULTURAL STUDIES',
    'CULTURAL STUDIES - AHCI;CULTURAL STUDIES - SSCI' : 'CULTURAL STUDIES',
    'CULTURAL STUDIES - SSCI':'CULTURAL STUDIES'
}

merged_all_df['Class'] = merged_all_df['Class'].replace(class_mapping)

                                              Class  Observations
0                                              ECON           372
1                                        LAW - ESCI           264
2                                  ECONOMICS - ESCI           155
3                                    ETHICS-CSR-MAN           145
4                                        LAW - SSCI           144
5                                           FINANCE           138
6                                 MANAGEMENT - ESCI           117
7                                  SOCIOLOGY - SSCI           113
8                                            SECTOR           112
9                                           SOC SCI           103
10                                         INFO MAN           102
11                                          ACCOUNT           100
12                                  BUSINESS - ESCI            90
13                                              MKT            87
14        

In [138]:
merged_all_df['Class'].unique()

array([nan, 'CABS ACCOUNT', 'CABS ETHICS-CSR-MAN', 'ECONOMICS',
       'SOCIOLOGY', 'PUBLIC ADMINISTRATION', 'LAW', 'CABS PSYCH (WOP-OB)',
       'BUSINESS', 'CABS ECON', 'CABS SOC SCI',
       'CABS BUS HIST & ECON HIST', 'HEALTH POLICY & SERVICES',
       'PSYCHOLOGY, APPLIED', 'CABS OR&MANSCI',
       'CABS REGIONAL STUDIES, PLANNING AND ENVIRONMENT', 'MANAGEMENT',
       'BUSINESS, FINANCE', 'CABS FINANCE', 'CABS IB&AREA',
       'CABS PUB SEC', 'CABS ORG STUD', 'CABS OPS&TECH', 'CABS HRM&EMP',
       'CABS MKT', 'PSYCHOLOGY, SOCIAL', 'CABS PSYCH (GENERAL)',
       'CULTURAL STUDIES', 'CABS SECTOR', 'CABS STRAT', 'CABS INNOV',
       'CABS ENT-SBM', 'CABS INFO MAN', 'CABS MDEV&EDU'], dtype=object)

In [142]:
# Save to Excel with separate sheets for each Class
excel_output_path = 'output file.xlsx'

# Create an ExcelWriter
with pd.ExcelWriter(excel_output_path, engine='xlsxwriter') as writer:
    # Get unique classes and handle NaN
    unique_classes = merged_all_df['Class'].fillna('unclassified').unique()

    # Define colors
    colors = {
        'light_gray': '#D3D3D3',  # Light Gray
        'purple': '#800080',       # Purple
        'light_blue': '#ADD8E6',   # Light Blue
        'salmon': '#FA8072',       # Salmon
        'black_white': '#000000'   # Black with White Letters
    }

    # Define formats for each color
    formats = {}
    workbook = writer.book
    for color, hex_code in colors.items():
        formats[color] = workbook.add_format({'font_color': 'white' if color == 'black_white' else 'black',
                                               'bg_color': hex_code})

    # Write each class to a separate sheet
    for class_name in unique_classes:
        # Limit the length of the class name
        sheet_name = 'unclassified' if pd.isna(class_name) else str(class_name)[:31]

        # Filter rows for each class
        class_df = merged_all_df[merged_all_df['Class'].fillna('unclassified') == class_name]

        # Write to Excel sheet
        class_df.to_excel(writer, sheet_name=sheet_name, index=False)

        # Apply color formatting to header row
        worksheet = writer.sheets[sheet_name]
        for col_num, col_name in enumerate(class_df.columns):
            color_format = None
            if col_name in ['Class','JCR Class', 'issn', 'e-issn', 'Journal Title (CABS)', 'Tytuł 1','AJG 2015', 'AJG 2018', 'AJG 2021']:
                color_format = formats['light_gray']
            elif col_name in ['2022 JIF (in 2022)', 'JIF Quartile (in 2022)', 'JIF Rank (in 2022)',
                              '5 Year JIF (in 2022)', '5 Year JIF Quartile (in 2022)', '2022 JCI (in 2022)',
                              'JCI Rank (in 2022)', 'JCI Quartile (in 2022)']:
                color_format = formats['purple']
            elif col_name in ['2021 JIF (in 2021)', 'JIF Quartile (in 2021)', '5 Year JIF (in 2021)',
                              'JIF Without Self Cites (in 2021)', 'JIF Percentile (in 2021)', '2021 JCI (in 2021)',
                              'JCI Percentile (in 2021)']:
                color_format = formats['light_blue']
            elif col_name in ['Points Feb21', 'Points Dec21', 'Points Jan24']:
                color_format = formats['salmon']
            elif col_name in ['ekonomia i finance', 'nauki o zarządzaniu i jakości', 'Publisher Name']:
                color_format = formats['black_white']
            if color_format:
                worksheet.write(0, col_num, col_name, color_format)

    # Write 'all' sheet with entire merged_all_df data
    merged_all_df.to_excel(writer, sheet_name='all', index=False)

    # Apply color formatting to header row of 'all' sheet
    all_worksheet = writer.sheets['all']
    for col_num, col_name in enumerate(merged_all_df.columns):
        color_format = None
        if col_name in ['Class','JCR Class', 'issn', 'e-issn', 'Journal Title (CABS)', 'Tytuł 1','AJG 2015', 'AJG 2018', 'AJG 2021']:
            color_format = formats['light_gray']
        elif col_name in ['2022 JIF (in 2022)', 'JIF Quartile (in 2022)', 'JIF Rank (in 2022)',
                          '5 Year JIF (in 2022)', '5 Year JIF Quartile (in 2022)', '2022 JCI (in 2022)',
                          'JCI Rank (in 2022)', 'JCI Quartile (in 2022)']:
            color_format = formats['purple']
        elif col_name in ['2021 JIF (in 2021)', 'JIF Quartile (in 2021)', '5 Year JIF (in 2021)',
                          'JIF Without Self Cites (in 2021)', 'JIF Percentile (in 2021)', '2021 JCI (in 2021)',
                          'JCI Percentile (in 2021)']:
            color_format = formats['light_blue']
        elif col_name in ['Points Feb21', 'Points Dec21', 'Points Jan24']:
            color_format = formats['salmon']
        elif col_name in ['ekonomia i finance', 'nauki o zarządzaniu i jakości', 'Publisher Name']:
            color_format = formats['black_white']
        if color_format:
            all_worksheet.write(0, col_num, col_name, color_format)

# Save the Excel file
writer.save()


C:\Users\Alon\AppData\Local\Temp\ipykernel_8404\3057089194.py:82: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
c:\Python311\Lib\site-packages\xlsxwriter\workbook.py:368: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")


### 2nd Task - JCR 2022

In [159]:
# Define file paths
file_2022 = 'source files/JCR_2022_ALL.xlsx'
file_2021 = 'source files/JCR_2021_ALL.xlsx'

# Load Excel file for 2022 into a DataFrame
jcr_2022_df = pd.read_excel(file_2022, sheet_name='ALL')

# Load Excel file for 2021, concatenating all sheets into a single DataFrame
jcr_2021_sheets = pd.read_excel(file_2021, sheet_name=None)
jcr_2021_df = pd.concat(jcr_2021_sheets.values(), ignore_index=True)

In [162]:
# Check for duplicate ISSN values in jcr_2022_df based on ISSN and Category
duplicate_2022 = jcr_2022_df[jcr_2022_df.duplicated(subset=['ISSN', 'Category'], keep=False)]
print("Duplicate ISSN values in jcr_2022_df:", len(duplicate_2022))

# Check for duplicate ISSN values in jcr_2021_df based on ISSN and Category
duplicate_2021 = jcr_2021_df[jcr_2021_df.duplicated(subset=['ISSN', 'Category'], keep=False)]
print("Duplicate ISSN values in jcr_2021_df:", len(duplicate_2021))

Duplicate ISSN values in jcr_2022_df: 63
Duplicate ISSN values in jcr_2021_df: 44


In [171]:
# Fill missing values in ISSN from eISSN to allow for the join function to find unique matches
# Fill missing values in 'ISSN' from 'eISSN' in jcr_2022_df
jcr_2022_df['ISSN'].fillna(jcr_2022_df['eISSN'], inplace=True)

# Fill missing values in 'ISSN' from 'eISSN' in jcr_2021_df
jcr_2021_df['ISSN'].fillna(jcr_2021_df['eISSN'], inplace=True)

Extend jcr_2022_df by the other dataframes


In [176]:
### 2022 with 2021
# Merge 2021 data to 2022 based on ISSN and Category columns, keeping only observations from 2022
jcr_extended = pd.merge(jcr_2022_df, jcr_2021_df, on=['ISSN', 'Category'], how='left', suffixes=('_2022', '_2021'))

# Select and rename columns
jcr_extended = jcr_extended[['Journal name_2022', 'ISSN', 'eISSN_2022', 'Category', '2022 JIF', 'JIF Quartile_2022',
                             'JIF Rank', '5 Year JIF_2022', '5 Year JIF Quartile', '2022 JCI', 'JCI Rank',
                             'JCI Quartile', '2021 JIF', 'JIF Quartile_2021', '5 Year JIF_2021',
                             'JIF Without Self Cites', 'JIF Percentile', '2021 JCI', 'JCI Percentile']]

jcr_extended.columns = ['Journal name', 'issn', 'e-issn', 'Category', 'JIF_2022', 'JIF Quartile_2022',
                        'JIF Rank_2022', '5 Year JIF_2022', '5 Year JIF Quartile_2022', 'JCI_2022', 'JCI Rank_2022',
                        'JCI Quartile_2022', '2021 JIF_2022', 'JIF Quartile_2021', '5 Year JIF_2021',
                        'JIF Without Self Cites_2021', 'JIF Percentile_2021', 'JCI_2021', 'JCI Percentile_2021']



In [192]:
### jcr_extended with wykaz
# Share value of points between all observations
wykaz_df_deduplicated = wykaz_df_deduplicated.groupby(['issn']).apply(lambda group: group.ffill().bfill())

# Deduplicate based on the 'ISSN' column
wykaz_df_deduplicated = wykaz_df_deduplicated.drop_duplicates(subset=['issn'])

# Merge wykaz_df_deduplicated data to jcr_extended based on ISSN and Category columns, keeping only observations from 2022
jcr_wykaz = pd.merge(jcr_extended, wykaz_df_deduplicated, on=['issn'], how='left')



C:\Users\Alon\AppData\Local\Temp\ipykernel_8404\4203177485.py:3: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  wykaz_df_deduplicated = wykaz_df_deduplicated.groupby(['issn']).apply(lambda group: group.ffill().bfill())


In [195]:
# jcr_wykaz extended with CABS
jcr_wykaz_cabs = pd.merge(jcr_wykaz, cabs_df_selected, left_on=['issn'], right_on=['e-issn'], how='left')

# List of columns to drop
columns_to_drop = ['e-issn_y', 'Class', 'Journal Title (CABS)', 'Publisher Name']

# Drop the columns
jcr_wykaz_cabs.drop(columns=columns_to_drop, inplace=True)

# Rename the e-issn column
jcr_wykaz_cabs.rename(columns={'e-issn_x': 'e-issn'}, inplace=True)

In [198]:
# Save the file back to an excel output
# Dictionary to map Category to sheet names
category_sheet_mapping = {
    'BUSINESS, FINANCE - SSCI': 'BUSINESS, FINANCE',
    'BUSINESS, FINANCE - ESCI': 'BUSINESS, FINANCE',
    'BUSINESS - SSCI': 'BUSINESS',
    'BUSINESS - ESCI': 'BUSINESS',
    'CULTURAL STUDIES - ESCI': 'CULTURAL STUDIES',
    'CULTURAL STUDIES - SSCI': 'CULTURAL STUDIES',
    'ECONOMICS - SSCI': 'ECONOMICS',
    'ECONOMICS - ESCI': 'ECONOMICS',
    'HEALTH POLICY & SERVICES - SSCI': 'HEALTH POLICY & SERVICES',
    'HEALTH POLICY & SERVICES - ESCI': 'HEALTH POLICY & SERVICES',
    'LAW - ESCI': 'LAW',
    'LAW - SSCI': 'LAW',
    'MANAGEMENT - SSCI': 'MANAGEMENT',
    'MANAGEMENT - ESCI': 'MANAGEMENT',
    'PSYCHOLOGY, APPLIED - SSCI': 'PSYCHOLOGY, APPLIED',
    'PSYCHOLOGY, APPLIED - ESCI': 'PSYCHOLOGY, APPLIED',
    'PSYCHOLOGY, SOCIAL - ESCI': 'PSYCHOLOGY, SOCIAL',
    'PSYCHOLOGY, SOCIAL - SSCI': 'PSYCHOLOGY, SOCIAL',
    'PUBLIC ADMINISTRATION - SSCI': 'PUBLIC ADMINISTRATION',
    'PUBLIC ADMINISTRATION - ESCI': 'PUBLIC ADMINISTRATION', 
    'SOCIOLOGY - SSCI': 'SOCIOLOGY',
    'SOCIOLOGY - ESCI': 'SOCIOLOGY'
}

# Save to Excel with separate sheets for each Category
excel_output_path = 'jcr_wykaz_cabs.xlsx'

# Create an ExcelWriter
with pd.ExcelWriter(excel_output_path, engine='xlsxwriter') as writer:
    # Iterate over unique Categories and create sheets
    for category, sheet_name in category_sheet_mapping.items():
        # Filter rows for each Category
        category_df = jcr_wykaz_cabs[jcr_wykaz_cabs['Category'] == category]
        # Write to Excel sheet
        category_df.to_excel(writer, sheet_name=sheet_name, index=False)
    
    # Add additional sheet named "ALL" containing entire DataFrame
    jcr_wykaz_cabs.to_excel(writer, sheet_name='ALL', index=False)

